In [1]:
import heapq
import copy
import time

class Puzzle:
    def __init__(self, state, parent=None, move=None, depth=0, cost=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost
        self.empty_pos = self.find_empty()

    def find_empty(self):
        for i in range(4):
            for j in range(4):
                if self.state[i][j] == 0:
                    return (i, j)

    def get_neighbors(self):
        moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
        neighbors = []
        x, y = self.empty_pos
        for dx, dy in moves:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 4 and 0 <= ny < 4:
                new_state = copy.deepcopy(self.state)
                new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
                neighbors.append(Puzzle(new_state, self, (dx, dy), self.depth + 1))
        return neighbors

    def __lt__(self, other):
        return self.cost < other.cost  # Needed for heapq priority queue


def heuristic_manhattan(state, goal):
    """Optimized Manhattan Distance heuristic."""
    dist = 0
    misplaced = 0
    positions = {goal[i][j]: (i, j) for i in range(4) for j in range(4)}  # Precompute goal positions
    for i in range(4):
        for j in range(4):
            tile = state[i][j]
            if tile != 0:
                goal_x, goal_y = positions[tile]
                dist += abs(i - goal_x) + abs(j - goal_y)
                if (i, j) != (goal_x, goal_y):  # Count misplaced tiles
                    misplaced += 1
    return dist, misplaced


def astar(start, goal, heuristic):
    start_time = time.time()  # Track execution time

    open_set = []
    closed_set = set()
    start_node = Puzzle(start)
    goal_node = Puzzle(goal)
    start_node.cost, start_misplaced = heuristic(start, goal)

    heapq.heappush(open_set, (start_node.cost, start_node))
    nodes_explored = 0

    while open_set:
        _, current = heapq.heappop(open_set)
        nodes_explored += 1

        # Logging to check progress
        current_heuristic, current_misplaced = heuristic(current.state, goal)
        if nodes_explored % 1000 == 0:
            elapsed_time = time.time() - start_time
            print(f"Nodes Explored: {nodes_explored}, Depth: {current.depth}, Misplaced: {current_misplaced}, Time: {elapsed_time:.2f} sec")

        if current.state == goal_node.state:
            path = []
            while current:
                path.append(current)
                current = current.parent
            total_time = time.time() - start_time
            print(f"\nSolution Found! Moves: {len(path) - 1}, Nodes Explored: {nodes_explored}, Time: {total_time:.2f} sec\n")
            return path[::-1], nodes_explored  # Reverse to get correct order

        # Use frozenset to store closed states efficiently
        state_tuple = frozenset(tuple(map(tuple, current.state)))
        closed_set.add(state_tuple)

        for neighbor in current.get_neighbors():
            neighbor_tuple = frozenset(tuple(map(tuple, neighbor.state)))
            if neighbor_tuple in closed_set:
                continue
            neighbor.cost, neighbor_misplaced = heuristic(neighbor.state, goal)
            heapq.heappush(open_set, (neighbor.cost, neighbor))

    print("No solution found.")
    return None, nodes_explored


def print_puzzle(state):
    """Prints the puzzle grid in a readable format."""
    for row in state:
        print(" ".join(f"{num:2}" if num != 0 else " _" for num in row))
    print("-" * 20)


# Test Case
initial_state = [[11, 5, 13, 4],
                 [8, 6, 10, 2],
                 [12, 7, 3, 1],
                 [9, 14, 0, 15]]

goal_state = [[1, 2, 3, 4],
              [5, 6, 7, 8],
              [9, 10, 11, 12],
              [13, 14, 15, 0]]

# Solve with Manhattan Distance heuristic
path_manhattan, nodes_manhattan = astar(initial_state, goal_state, heuristic_manhattan)

# Print Solution Steps
if path_manhattan:
    print("\nSolution Path:")
    for i, step in enumerate(path_manhattan):
        print(f"Move {i} (Misplaced Tiles: {heuristic_manhattan(step.state, goal_state)[1]}):")
        print_puzzle(step.state)

# Compare Performance
print("\nPerformance Summary:")
print(f"Manhattan Distance -> Moves: {len(path_manhattan) - 1 if path_manhattan else 'N/A'}, Nodes Explored: {nodes_manhattan}")



Solution Found! Moves: 125, Nodes Explored: 729, Time: 0.05 sec


Solution Path:
Move 0 (Misplaced Tiles: 12):
11  5 13  4
 8  6 10  2
12  7  3  1
 9 14  _ 15
--------------------
Move 1 (Misplaced Tiles: 12):
11  5 13  4
 8  6 10  2
12  7  _  1
 9 14  3 15
--------------------
Move 2 (Misplaced Tiles: 12):
11  5 13  4
 8  6  _  2
12  7 10  1
 9 14  3 15
--------------------
Move 3 (Misplaced Tiles: 12):
11  5 13  4
 8  6  2  _
12  7 10  1
 9 14  3 15
--------------------
Move 4 (Misplaced Tiles: 12):
11  5 13  4
 8  6  2  1
12  7 10  _
 9 14  3 15
--------------------
Move 5 (Misplaced Tiles: 12):
11  5 13  4
 8  6  2  1
12  7  _ 10
 9 14  3 15
--------------------
Move 6 (Misplaced Tiles: 12):
11  5 13  4
 8  6  2  1
12  _  7 10
 9 14  3 15
--------------------
Move 7 (Misplaced Tiles: 12):
11  5 13  4
 8  6  2  1
 _ 12  7 10
 9 14  3 15
--------------------
Move 8 (Misplaced Tiles: 12):
11  5 13  4
 _  6  2  1
 8 12  7 10
 9 14  3 15
--------------------
Move 9 (Misplaced Tiles: 12